In [1]:
# Import all the required libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder as le
from sklearn import metrics as me
import time as t

# Reading of data

In [2]:
df = pd.read_csv("train_data.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- Load all the train data into a new data frame

In [3]:
df.head() # print the first top 5 rows of the dataset

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1


# Pre-Processing of data

- To make the final prediction better, try to get the best features from the data

In [4]:
details = []

for i in df.columns:
    details.append((i, df[i].nunique(), 
                  df[i].isnull().sum() * 100 / df.shape[0],   
                  df[i].value_counts(normalize=True, dropna=False).values[0] * 100,   
                  df[i].dtype))

- Iterate through each column in the data to analyse the impact of them on the final prediction
- find out the unique values in each column
- find out the percentage of missing values in each colum
- find out how much percentage of the values in a column tend towards one value

In [5]:
details_df = pd.DataFrame(details, columns=['Feature', 'Unique values', '% of Missing values', '% of Values in the biggest category', 'datatype'])

- Now create a data frame with the above details 
- Then sort them according to the percentage of missing values

In [6]:
# Sort them according to the missing values
details_df.sort_values('% of Missing values', ascending=False)

,Feature,Unique values,% of Missing values,% of Values in the biggest category,datatype
28,PuaMode,1,99.974828,99.974828,object
41,Census_ProcessorClass,3,99.579680,99.579680,object
8,DefaultBrowsersIdentifier,566,95.134711,95.134711,float64
68,Census_IsFlightingInternal,2,83.021931,83.021931,float64
52,Census_InternalBatteryType,28,71.028574,71.028574,object
71,Census_ThresholdOptIn,2,63.504187,63.504187,float64
75,Census_IsWIMBootEnabled,1,63.416571,63.416571,float64
31,SmartScreen,12,35.656540,48.345942,object
15,OrganizationIdentifier,43,30.867353,47.088495,float64
29,SMode,2,6.011899,93.944586,float64


In [7]:
final_columns = list(df.columns)

- Now create a copy of the columns in the dataframe

In [8]:
df.shape # Check the dimensionality of the dataframe

(512467, 83)

In [9]:
for i in df.columns:
    rate = df[i].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        final_columns.remove(i)

- Now iterate thorugh each column to find out the columns which are unbalanced.
- Unbalanced columns can be like columns which have more than 90% missing values (or) columns which have more than 90% of its values tend towards a single value.
- In DefaultBrowsersIdentifier column, more than 95% values tend towards to one value, so it might not be so helpful in prediction.
- Few columns like PuaMode and Census_ProcessorClass have 99% plus missing values, that shows that these columns are not that useful.
- Apart from the above 2 there are 24 other columns in which one category contains more than 90% values.

In [10]:
len(final_columns)

57

- Check the length of the final columns after removing unbalanced ones.
- So in total 26 columns are removed.

In [11]:
df = df[final_columns] # Update the data frame with the new columns

In [12]:
df['HasDetections'].value_counts() # Check the dimensionality of the dataframe

1    256696
0    255771
Name: HasDetections, dtype: int64

In [13]:
details= []

for i in df.columns:
    details.append((i, df[i].nunique(), 
                  df[i].isnull().sum() * 100 / df.shape[0],   
                  df[i].value_counts(normalize=True, dropna=False).values[0] * 100,   
                  df[i].dtype))

details_df = pd.DataFrame(details, columns=['Feature', 'Unique values', '% of Missing values', '% of Values in the biggest category', 'datatype'])

# Sort them according to the missing values
details_df.sort_values('% of Missing values', ascending=False)

,Feature,Unique values,% of Missing values,% of Values in the biggest category,datatype
47,Census_IsFlightingInternal,2,83.021931,83.021931,float64
33,Census_InternalBatteryType,28,71.028574,71.028574,object
48,Census_ThresholdOptIn,2,63.504187,63.504187,float64
52,Census_IsWIMBootEnabled,1,63.416571,63.416571,float64
17,SmartScreen,12,35.656540,48.345942,object
8,OrganizationIdentifier,43,30.867353,47.088495,float64
7,CityIdentifier,37755,3.639259,3.639259,float64
55,Wdft_RegionIdentifier,15,3.424611,20.184909,float64
54,Wdft_IsGamer,2,3.424611,69.260460,float64
34,Census_InternalBatteryNumberOfCharges,5289,3.027707,56.556422,float64


In [14]:
df.Census_IsFlightingInternal.value_counts()

0.0    87006
1.0        1
Name: Census_IsFlightingInternal, dtype: int64

In [15]:
df.Census_IsFlightingInternal.isnull().sum()

425460

- In Census_IsFlightingInternal more than 83% values are null and almost all the remaining are zeros. So its better to drop that column as we can't replace the null ones with zeros (if did then almost everything will be zero)

In [16]:
df = df.drop(columns=['Census_IsFlightingInternal'])

In [17]:
df.Census_InternalBatteryType.value_counts()

lion    116448
li-i     14118
#        10684
lip       3610
liio      1906
li p       460
li         379
nimh       263
real       150
pbac       132
bq20       125
vbox        88
unkn        24
lgi0        21
lipo        12
lipp        12
4cel        10
lhp0         6
batt         5
ithi         4
bad          3
virt         2
ram          2
a140         1
asmb         1
lit          1
lio         1
ca48         1
Name: Census_InternalBatteryType, dtype: int64

In [18]:
df.Census_InternalBatteryType.isnull().sum()

363998

In [19]:
exchange_dict = {'#': 'unknown', 'unkn': 'unknown', np.nan: 'unknown'}
df.replace({'Census_InternalBatteryType': exchange_dict}, inplace=True)

In [20]:
df.Census_InternalBatteryType.value_counts()

unknown    374706
lion       116448
li-i        14118
lip          3610
liio         1906
li p          460
li            379
nimh          263
real          150
pbac          132
bq20          125
vbox           88
lgi0           21
lipp           12
lipo           12
4cel           10
lhp0            6
batt            5
ithi            4
bad             3
ram             2
virt            2
a140            1
lio            1
lit             1
ca48            1
asmb            1
Name: Census_InternalBatteryType, dtype: int64

In [21]:
df.Census_ThresholdOptIn.value_counts()

0.0    186976
1.0        53
Name: Census_ThresholdOptIn, dtype: int64

In [22]:
df['Census_ThresholdOptIn'].isnull().sum()

325438

- In Census_ThresholdOptIn more than 60% values are null and almost all the remaining are zeros. So its better to drop that column as we can't replace the null ones with zeros (if did then almost everything will be zero)

In [23]:
df.Census_IsWIMBootEnabled.value_counts()

0.0    187478
Name: Census_IsWIMBootEnabled, dtype: int64

In [24]:
df['Census_IsWIMBootEnabled'].isnull().sum()

324989

- In Census_IsWIMBootEnabled more than 60% values are null and all the remaining are zeros. So its better to drop that column as we can't replace the null ones with zeros (if did then almost everything will be zero)

In [25]:
df.SmartScreen.value_counts()

RequireAdmin    247757
ExistsNotSet     60152
Off              10739
Warn              7691
Prompt            1956
Block             1266
off                 82
On                  38
&#x01;              24
&#x02;              22
on                  11
OFF                  1
Name: SmartScreen, dtype: int64

In [26]:
df['SmartScreen'].isnull().sum()

182728

In [27]:
df.OrganizationIdentifier.value_counts()

27.0    241313
18.0    100746
48.0      3691
50.0      2598
37.0      1140
11.0      1140
49.0       790
46.0       649
14.0       278
32.0       264
36.0       242
33.0       189
52.0       180
2.0        145
5.0        120
28.0       101
40.0        94
4.0         86
10.0        76
51.0        59
20.0        49
8.0         48
1.0         44
39.0        31
6.0         29
16.0        25
47.0        24
31.0        22
3.0         18
21.0        15
22.0        14
7.0         13
26.0        11
29.0         9
44.0         7
19.0         6
42.0         5
41.0         4
43.0         2
30.0         2
45.0         1
15.0         1
25.0         1
Name: OrganizationIdentifier, dtype: int64

In [28]:
df['OrganizationIdentifier'].isnull().sum()

158185

- Replace the null values in OrganizationIdentifier with zero

In [29]:
df.replace({'OrganizationIdentifier': {np.nan: 0}}, inplace=True)

In [30]:
df['OrganizationIdentifier'].isnull().sum()

0

In [31]:
df.corr()

,AVProductStatesIdentifier,AVProductsInstalled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,OsBuild,OsSuite,IeVerIdentifier,...,Census_OSUILocaleIdentifier,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsTouchEnabled,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
AVProductStatesIdentifier,1.000000,-0.633136,0.006302,-0.004190,0.030174,0.018567,-0.078963,0.002124,-0.054823,-0.071017,...,0.015203,0.003824,-0.013357,-0.016174,-0.034676,NaN,0.040474,-0.022451,-0.021202,0.116479
AVProductsInstalled,-0.633136,1.000000,-0.022182,0.004025,-0.006501,-0.048845,0.040843,0.002982,0.181002,-0.030721,...,0.008194,-0.007818,0.057026,0.041167,0.215271,NaN,-0.011677,0.015106,0.032799,-0.149067
CountryIdentifier,0.006302,-0.022182,1.000000,0.001184,-0.004946,0.596728,0.049384,0.010387,-0.001287,0.003994,...,0.097584,-0.001650,-0.020025,0.019835,-0.011070,NaN,0.003164,0.018696,-0.115136,0.006744
CityIdentifier,-0.004190,0.004025,0.001184,1.000000,-0.013316,-0.059694,0.025430,0.012357,0.009768,0.014545,...,0.090053,0.000288,-0.025970,0.013718,0.010611,NaN,-0.010455,0.003409,-0.057381,-0.006247
OrganizationIdentifier,0.030174,-0.006501,-0.004946,-0.013316,1.000000,-0.024801,0.027335,-0.013141,0.031996,0.012830,...,-0.021662,0.003828,-0.002522,0.004576,0.021876,NaN,0.012479,-0.011007,0.058638,0.005244
GeoNameIdentifier,0.018567,-0.048845,0.596728,-0.059694,-0.024801,1.000000,-0.036697,-0.016794,-0.057665,-0.013136,...,-0.120922,0.001896,-0.000779,0.005693,-0.065355,NaN,0.020783,0.001659,-0.023931,0.004039
LocaleEnglishNameIdentifier,-0.078963,0.040843,0.049384,0.025430,0.027335,-0.036697,1.000000,-0.006100,0.063816,0.009221,...,0.119973,-0.004795,0.008962,0.016682,0.024893,NaN,-0.054999,-0.003444,0.162363,0.016613
OsBuild,0.002124,0.002982,0.010387,0.012357,-0.013141,-0.016794,-0.006100,1.000000,-0.032592,-0.115413,...,0.019163,0.004733,-0.063069,-0.016189,-0.002107,NaN,-0.024070,0.141933,-0.026418,0.025751
OsSuite,-0.054823,0.181002,-0.001287,0.009768,0.031996,-0.057665,0.063816,-0.032592,1.000000,-0.008669,...,0.007774,-0.016105,0.078102,0.131439,0.456544,NaN,0.093309,-0.015065,0.036606,-0.021931
IeVerIdentifier,-0.071017,-0.030721,0.003994,0.014545,0.012830,-0.013136,0.009221,-0.115413,-0.008669,1.000000,...,0.035499,0.010545,-0.018214,-0.015980,-0.009310,NaN,-0.022112,-0.040391,0.003995,0.016323
